In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from ast import literal_eval
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from collections import defaultdict

Read each fitting results file and print the fitting criteria and mean parameters

In [2]:
DATA_DIR = "../../data/fitting_results"
FIGURE_DIR = "../../figures"

In [3]:
# create a dictionary to store dataframes
all_model_dfs = {}
for agent_type in ['lqr', 'sparse_lqr', 'sparse_max_discrete', 'sparse_max_continuous', 'null_model_1', 'null_model_2', 'hill_climbing']:
    # read the aggregated fitting results
    df = pd.read_csv(f"{DATA_DIR}/fitting_results_{agent_type}.csv")
    all_model_dfs[agent_type] = df
    # print info about 
    print(f"agent type: {agent_type}, n={len(df)}")
    print(f"{agent_type} llh: {df['ll'].sum()}")
    print(f"{agent_type} AIC: {df['AIC'].sum()}")
    print(f"{agent_type} llh mean: {df['ll'].mean()}")
    print(f"{agent_type} AIC mean: {df['AIC'].mean()}")
    if agent_type == "lqr":
        print(f"mean exp param: {df['exp_param'].mean()}")
        print(f"mean vm param: {df['vm_param'].mean()}")

agent type: lqr, n=111
lqr llh: -12814.148508
lqr AIC: 26072.297016
lqr llh mean: -115.44277935135132
lqr AIC mean: 234.8855587027027
mean exp param: 0.00940909749817091
mean vm param: 2.020889219661038
agent type: sparse_lqr, n=111
sparse_lqr llh: -8979.656426999998
sparse_lqr AIC: 18625.312853999996
sparse_lqr llh mean: -80.89780564864864
sparse_lqr AIC mean: 167.79561129729726
agent type: sparse_max_discrete, n=111
sparse_max_discrete llh: -8207.513571
sparse_max_discrete AIC: 17303.027142
sparse_max_discrete llh mean: -73.9415637027027
sparse_max_discrete AIC mean: 155.8831274054054
agent type: sparse_max_continuous, n=111
sparse_max_continuous llh: -8387.806881
sparse_max_continuous AIC: 17663.613762
sparse_max_continuous llh mean: -75.56582775675678
sparse_max_continuous AIC mean: 159.13165551351355
agent type: null_model_1, n=111
null_model_1 llh: -8869.6115108
null_model_1 AIC: 18627.2230216
null_model_1 llh mean: -79.9064100072072
null_model_1 AIC mean: 167.81282001441443
agen

In [4]:
# read the participant ids
pp_nrs = pd.read_csv('../../data/experimental_data/experiment_ppids.csv')['id']

Get the number of participants best fit by each model, as well as the strength of evidence for the best model over the second-best model for each participant

In [5]:
models_by_best_fitting_pps = defaultdict(int)
participant_to_best_model = {}

for pp_id in pp_nrs:
    participant_fits = {}
    for agent_type in ['lqr', 'sparse_lqr', 'sparse_max_discrete', 'sparse_max_continuous', 'null_model_1', 'null_model_2', 'hill_climbing']:
        # select the dataframe for the selected agent type
        df = all_model_dfs[agent_type]

        if len(df[df['pp_id'] == pp_id]['AIC']) != 1:
            print("len:", len(df[df['pp_id'] == pp_id]['AIC']))
            break
        
        # add the AIC to the participant fits dictionary
        participant_fits[agent_type] = float(df[df['pp_id'] == pp_id]['AIC'])
    
    if len(df[df['pp_id'] == pp_id]) != 1:
        continue
    
    # select the best-fitting agent for this participant
    sorted_fits = sorted(participant_fits.values())
    best_agent = min(participant_fits, key=participant_fits.get)
    
    # increment the number of pps best fit by the model
    models_by_best_fitting_pps[best_agent] += 1
    # store the best-fitting model for this participant
    participant_to_best_model[pp_id] = best_agent

print(models_by_best_fitting_pps)

defaultdict(<class 'int'>, {'null_model_2': 17, 'hill_climbing': 35, 'sparse_max_discrete': 33, 'sparse_lqr': 15, 'sparse_max_continuous': 10, 'lqr': 1})


Create csv files for Bayesian model selection (done using SPM8)

In [6]:
df_aics = pd.DataFrame()
for df_type in all_model_dfs:
    # convert AICs to log model evidence format
    df_aics[df_type] = all_model_dfs[df_type]['AIC'].apply(lambda x: -x/2)
df_aics.to_csv(f"{DATA_DIR}/aic_lme.csv")  # save to csv

n_params = {"null_model_2": 2, "null_model_1": 4, "lqr": 2, "sparse_lqr": 3, "hill_climbing": 3, "sparse_max_continuous": 4, "sparse_max_discrete": 4}

df_bics = pd.DataFrame()
for df_type in all_model_dfs:
    # convert BICs to log model evidence format
    df_bics[df_type] = all_model_dfs[df_type]["ll"].apply(lambda x: - (n_params[df_type] * np.log(10) - 2 * x) / 2)
df_bics.to_csv(f"{DATA_DIR}/bic_lme.csv")  # save to csv

In [7]:
# make a csv file with the best-fitting model and parameters for each participant
df_bestfit = pd.DataFrame()
for pp_id in pp_nrs:
    best_model_type = participant_to_best_model[pp_id]  # pick the best model type
    df = all_model_dfs[best_model_type]  # get the dataframe for the best model
    row = df[df['pp_id'] == pp_id]  # get the relevant row using the pp id
    df_bestfit = df_bestfit.append(row, ignore_index=True)  # add the selected row to the best fit dataframe
df_bestfit.to_csv(f"{DATA_DIR}/best_fitting_models.csv")

Get the mean best-fitting parameter for each model type

In [8]:
for model_type in df_bestfit['agent_type'].drop_duplicates():
    # get only the rows of the best-fit dataframe for the current model type
    df_model = df_bestfit[df_bestfit['agent_type'] == model_type]
    print(f"MODEL TYPE: {model_type}")
    print(f"exp param: {df_model['exp_param'].mean()}")
    print(f"vm param: {df_model['vm_param'].mean()}")
    if model_type in ("hill_climbing", "sparse_max_continuous", "sparse_max_discrete"):
        print(f"step size: {df_model['step_size'].mean()}")
    if model_type in ("sparse_lqr", "sparse_max_continuous", "sparse_max_discrete"):
        print(f"attention cost: {df_model['attention_cost'].mean()}")
    if model_type == "null_model_1":  # n and b parameters for nm1
        print(f"n: {np.round(df_model['n']).mean()}")
        print(f"b: {df_model['b'].mean()}")

MODEL TYPE: null_model_2
exp param: 0.02613745480687401
vm param: 5.610590213263846
MODEL TYPE: hill_climbing
exp param: 0.06961741245647332
vm param: 5.888150693552178
step size: 0.3845915454380134
MODEL TYPE: sparse_max_discrete
exp param: 0.09070498285942245
vm param: 4.406183043702149
step size: 0.6362588893687267
attention cost: 12.433202622236541
MODEL TYPE: sparse_lqr
exp param: 0.051042430933889826
vm param: 4.445905714461947
attention cost: 136.06803730850126
MODEL TYPE: sparse_max_continuous
exp param: 0.05548931159039938
vm param: 5.265772347464798
step size: 0.2071716908934227
attention cost: 14.49441125346252
MODEL TYPE: lqr
exp param: 0.025278183757093
vm param: 4.268855736237463


## Compare Participant Scores by Best-Fitting Model

In [9]:
# read the participant data
raw_pp_data_path = '../../data/experimental_data/experiment_actions.csv'
df_pps = pd.read_csv(raw_pp_data_path)
df_last = df_pps.loc[df_pps.groupby("pp_id")['Unnamed: 0'].idxmax()]

Get scores by which model explains each pp best

In [10]:
scores_by_best_model = defaultdict(list)
for index, row in df_last.iterrows():
    scores_by_best_model[participant_to_best_model[row['pp_id']]].append(np.sqrt(row['total_cost']))

In [11]:
print(f"sparse lqr median: {np.median(scores_by_best_model['sparse_lqr'])}")
print(f"hill climbing median: {np.median(scores_by_best_model['hill_climbing'])}")

sparse lqr median: 90.62030677502698
hill climbing median: 115.66256957200979


In [12]:
def bootstrap_ci(data, n=1000000):
    all_medians = []
    for i in range(n):
        bs_data = np.random.choice(data, len(data), replace=True)
        med = np.median(bs_data)
        all_medians.append(med)
    all_medians = np.array(all_medians)
    lower_bound = np.percentile(all_medians, 2.5)
    upper_bound = np.percentile(all_medians, 97.5)
    
    return lower_bound, upper_bound

In [13]:
for model_type in ("sparse_lqr", "hill_climbing"):
    scores = scores_by_best_model[model_type]
    lower_bound, upper_bound = bootstrap_ci(scores)
    print(f"{model_type}: {np.median(scores)}, [{lower_bound}, {upper_bound}]")

sparse_lqr: 90.62030677502698, [26.370058778849923, 172.33374596984768]
hill_climbing: 115.66256957200979, [99.15694630231407, 129.1154522123514]


In [14]:
stats.kruskal(scores_by_best_model['sparse_lqr'], scores_by_best_model['hill_climbing'])

KruskalResult(statistic=1.0112044817927028, pvalue=0.3146144552999878)

Get descriptive stats like number of variables manipulated.

In [15]:
print(len(scores_by_best_model['sparse_lqr']) + len(scores_by_best_model['hill_climbing']))

50


Compute the number of variables manipulated and input norm standard deviation for humans.